In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [0]:
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/train.tsv',delimiter='\t',encoding='utf-8')
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/test.tsv',delimiter='\t',encoding='utf-8')

In [3]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
test_data.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [5]:
train = np.array(train_data.drop(["PhraseId","SentenceId"], axis = 1))
test = np.array(test_data.drop(["PhraseId","SentenceId"], axis = 1))
print(train)
print(test)

[['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'
  1]
 ['A series of escapades demonstrating the adage that what is good for the goose'
  2]
 ['A series' 2]
 ...
 ['avuncular chortles' 3]
 ['avuncular' 2]
 ['chortles' 2]]
[['An intermittently pleasing but mostly routine effort .']
 ['An intermittently pleasing but mostly routine effort']
 ['An']
 ...
 ['A long-winded ,']
 ['A long-winded']
 ['predictable scenario']]


In [0]:
train_sentences = []
train_labels = []
test_sentences = []
test_labels = []

for i in range(train.shape[0]):
  train_sentences.append(train[i, 0])
  train_labels.append(train[i, 1])
train_sentences_final = np.array(train_sentences)
train_labels_final = np.array(train_labels)

for j in range(test.shape[0]):
  test_sentences.append(test[j, 0])
test_sentences_final = np.array(test_sentences)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [8]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[100]))
print(train_sentences[100])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? would have a hard time sitting through this one
would have a hard time sitting through this one .


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               983552    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2

In [10]:
num_epochs = 10
model.fit(padded, train_labels_final, epochs=num_epochs)

Epoch 1/10
4877/4877 [==============================] - 21s 4ms/step - loss: 1.0487 - accuracy: 0.5769
Epoch 2/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.8137 - accuracy: 0.6705
Epoch 3/10
4877/4877 [==============================] - 22s 4ms/step - loss: 0.7412 - accuracy: 0.6983
Epoch 4/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.6941 - accuracy: 0.7165
Epoch 5/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.6578 - accuracy: 0.7302
Epoch 6/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.6299 - accuracy: 0.7411
Epoch 7/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.6067 - accuracy: 0.7498
Epoch 8/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.5843 - accuracy: 0.7565
Epoch 9/10
4877/4877 [==============================] - 21s 4ms/step - loss: 0.5653 - accuracy: 0.7635
Epoch 10/10
4877/4877 [==============================] - 22s 4ms/step - l

In [0]:
predicted_labels = model.predict(padded)

In [12]:
np.around(predicted_labels)

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)